In [0]:
# Import libraries

import helpr
import pandas as pd

In [0]:
def init_weather(city, years):
    
    # Read temperature and weather data files
    
    df_temp, df_wea = helpr.read_file(['temperature.csv', 'weather_description.csv'])
    
    # Extract temperature and weather data for city
    
    mtl_cols = ['datetime', city]
    df_temp, df_wea = helpr.extract_col_data([(df_temp, mtl_cols), (df_wea, mtl_cols)])
    df_temp[city] = df_temp[city] - 273.15 # Kelvin to Celsius
    
    # Merge city temperature and weather data
    
    df_temp, df_wea = helpr.to_dt([(df_temp,['datetime']), (df_wea,['datetime'])])
    df_temp_wea = pd.merge(df_temp, df_wea, on='datetime')
    df_temp_wea = df_temp_wea[df_temp_wea.datetime.dt.year>years]
    df_temp_wea.columns = ['datetime','temperature','weather']
    
    print('Weather data initialization complete.')

    return df_temp_wea

In [0]:
def init_bixi(city, years, datafiles):
    
    # Get Montreal temperature and weather data, from hereon referred to as just weather data
    df_wea = init_weather(city, years)
    
    list_df = []
    
    for file in datafiles:
        
        [df_bixi] = helpr.read_file([file])
        [df_bixi] = helpr.to_dt([(df_bixi, ['start_date', 'end_date'])])
        
        df_bixi = df_bixi.sort_values(by = ['start_date'])
        df_bixi = pd.merge_asof(df_bixi, df_wea, left_on = 'start_date', right_on = 'datetime', direction = 'nearest').drop('datetime',  axis=1)
        
        [df_bixi] = helpr.to_dt([(df_bixi, ['start_date'])])
        df_bixi['year'] = df_bixi.start_date.dt.year
        df_bixi['month'] = df_bixi.start_date.dt.month
        df_bixi['week_num'] = df_bixi.start_date.dt.weekofyear
        df_bixi['day_of_week'] = df_bixi.start_date.dt.dayofweek
        df_bixi['hour'] = df_bixi.start_date.dt.hour
    
        list_df.append(df_bixi)
        
    print('Bixi data initialization complete.')
        
    return list_df